In [1]:
from myscripts import defi_stats
from tools import data_preparation as dp
from postgres import executions

import pandas as pd
import numpy as np
import requests
import datetime
import os
import sys

import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)

<font size="6">DEFI - DAPPS</font>

In [2]:
save_in_db = True
tvl_threshold = '500 000'

In [3]:
defi_dapps_object = defi_stats.DefiDappsStats()

2023-10-21 21:44:03. DefiDappsStats: 2023-10-21 20:42:32 - 2023-10-21 21:44:02.
2023-10-21 21:44:03. Timedelta: 0 days 1 hours 1 minutes 30 seconds.
2023-10-21 21:44:03. Categories not included: ['CEX', 'Chain'].
2023-10-21 21:44:03. DefiDappsStats: current market data saved in db.
2023-10-21 21:44:03. DefiDappsStats: stats saved in db.
2023-10-21 21:44:03. Report created in: 0:00:01.794292.


In [4]:
defi_dapps_object.check_for_new_protocols()

,name,symbol,gecko_id,category,tvl,mcap,chains,chainTvls,date


In [5]:
defi_dapps = defi_dapps_object.stats

In [6]:
defi_dapps.head()

,symbol,name,gecko_id,category,tvl,tvl_y,mcap,tvl_perc,mcap_perc,chains,chainTvls,date,date_y
0,LDO,Lido,lido-dao,Liquid Staking,"14,562,927,058.77196","14,498,104,466.73077","1,467,931,356.39881",0.44711,0.48380,"[Ethereum, Solana, Moonbeam, Moonriver, Terra]","{'Ethereum': 14498044506.571108, 'Moonriver': ...",2023-10-21 21:44:02,2023-10-21 20:42:32
1,-,WBTC,0.00000,Bridge,"4,911,788,259.42740","4,880,620,193.23387",0.00000,0.63861,0.00000,"[Ethereum, Tron]","{'Ethereum': 4905520136.45989, 'Tron': 6268122...",2023-10-21 21:44:02,2023-10-21 20:42:32
2,JST,JustLend,just,Lending,"4,863,348,539.47568","4,858,560,520.57573","221,266,622.32716",0.09855,0.26671,[Tron],"{'Tron-borrowed': 87380945.01971692, 'Tron': 4...",2023-10-21 21:44:02,2023-10-21 20:42:32
3,MKR,MakerDAO,maker,CDP,"4,345,272,617.79870","4,329,997,050.58132","1,287,038,309.04078",0.35278,-0.37210,[Ethereum],{'Ethereum': 4345272617.798698},2023-10-21 21:44:02,2023-10-21 20:42:32
4,JST,JustCryptos,0.00000,Bridge,"3,428,059,009.00000","3,409,674,113.00000",0.00000,0.53920,0.00000,[Tron],{'Tron': 3428059009},2023-10-21 21:44:02,2023-10-21 20:42:32


In [7]:
defi_dapps_with_threshold = defi_dapps[defi_dapps.tvl >= dp.string_number_to_integer(tvl_threshold)]

<font size="5">TVL_CHANGE</font>

In [8]:
tvl_perc = defi_dapps_with_threshold.sort_values('tvl_perc', ascending=False).reset_index(drop=True).head(10)
tvl_perc

,symbol,name,gecko_id,category,tvl,tvl_y,mcap,tvl_perc,mcap_perc,chains,chainTvls,date,date_y
0,MNGO,Mango Markets V4,0.00000,Lending,"2,969,676.09434","2,726,527.04955",0.00000,8.91790,0.00000,[Solana],"{'Solana-borrowed': 1258435.0723867337, 'Solan...",2023-10-21 21:44:02,2023-10-21 20:42:32
1,NIIFI,NiiFi,niifi,Dexes,"898,068.91400","844,185.27135",0.00000,6.38292,0.00000,[Nahmii],{'Nahmii': 898068.9140018877},2023-10-21 21:44:02,2023-10-21 20:42:32
2,BRIDGE,CrossChain Bridge,cross-chain-bridge-token,Cross Chain,"3,055,642.73801","2,893,316.95011",0.00000,5.61037,0.00000,"[Ethereum, Binance, Polygon, Fantom, Avalanche]","{'Fantom-staking': 89.71114768827086, 'Binance...",2023-10-21 21:44:02,2023-10-21 20:42:32
3,MVX,Metavault.Trade,metavault-trade,Derivatives,"1,819,232.22819","1,740,450.43739","2,537,300.74435",4.52652,0.10943,[Polygon],"{'Polygon': 1819232.2281911129, 'Polygon-staki...",2023-10-21 21:44:02,2023-10-21 20:42:32
4,-,Levvy for Tokens,0.00000,Lending,"560,409.52596","548,933.35086",0.00000,2.09063,0.00000,[Cardano],"{'Cardano': 560409.5259577588, 'Cardano-borrow...",2023-10-21 21:44:02,2023-10-21 20:42:32
5,-,ZeroLend,0.00000,Lending,"940,998.56261","924,375.09704",0.00000,1.79835,0.00000,[zkSync Era],"{'zkSync Era': 940998.5626075879, 'zkSync Era-...",2023-10-21 21:44:02,2023-10-21 20:42:32
6,-,Filet Finance,0.00000,Liquid Staking,"6,086,476.03183","5,980,856.98012",0.00000,1.76595,0.00000,"[Filecoin, Binance, Mixin, Heco]","{'Binance': 519266.88706031995, 'Heco': 4575.6...",2023-10-21 21:44:02,2023-10-21 20:42:32
7,-,Skydrome,0.00000,Dexes,"1,457,822.79340","1,433,092.85322",0.00000,1.72563,0.00000,[Scroll],{'Scroll': 1457822.7934023011},2023-10-21 21:44:02,2023-10-21 20:42:32
8,ARBIS,Arbi's Finance,arbis-finance,Yield,"1,455,785.06471","1,431,192.11171",0.00000,1.71835,0.00000,[Arbitrum],"{'Arbitrum-staking': 0, 'Arbitrum': 1455785.06...",2023-10-21 21:44:02,2023-10-21 20:42:32
9,-,WEMIX.FI Staking,0.00000,Staking Pool,"150,775,284.00983","148,274,379.81800",0.00000,1.68667,0.00000,[WEMIX],{'WEMIX': 150775284.0098327},2023-10-21 21:44:02,2023-10-21 20:42:32


<font size="6">TVL_PER_CHAIN</font>

In [9]:
save_in_db = True
tvl_per_chain_threshold = '500 000'

In [10]:
tvl_per_chain = defi_stats.TvlPerChainStats(save_in_db).stats

2023-10-21 21:44:03. TvlPerChainStats: 2023-10-21 20:42:33 - 2023-10-21 21:44:03.
2023-10-21 21:44:03. Timedelta: 0 days 1 hours 1 minutes 30 seconds.
2023-10-21 21:44:04. TvlPerChainStats: current market data saved in db.
2023-10-21 21:44:04. TvlPerChainStats: stats saved in db.
2023-10-21 21:44:04. Report created in: 0:00:00.464395.


<font size="5">TOP #10 TVLs</font>

In [11]:
tvl_per_chain.sort_values('tvl', ascending=False).reset_index(drop=True).head(10)

,tokenSymbol,name,gecko_id,tvl,tvl_y,share,share_y,tvl_perc,share_perc,date,date_y
0,ETH,Ethereum,ethereum,"46,770,191,676.11198","46,628,640,192.57755",0.70032,0.69980,0.30357,0.07337,2023-10-21 21:44:03,2023-10-21 20:42:33
1,TRON,Tron,tron,"6,932,967,821.48392","6,929,046,620.04247",0.10381,0.10399,0.05659,-0.17305,2023-10-21 21:44:03,2023-10-21 20:42:33
2,BNB,Binance,binancecoin,"3,376,989,724.07890","3,370,854,944.76532",0.05057,0.05059,0.18199,-0.04793,2023-10-21 21:44:03,2023-10-21 20:42:33
3,ARB,Arbitrum,arbitrum,"1,974,692,388.68746","1,971,654,337.33462",0.02957,0.02959,0.15409,-0.07578,2023-10-21 21:44:03,2023-10-21 20:42:33
4,MATIC,Polygon,matic-network,"877,373,665.80424","876,561,889.92018",0.01314,0.01316,0.09261,-0.13711,2023-10-21 21:44:03,2023-10-21 20:42:33
5,SOL,Solana,solana,"752,680,033.83062","753,614,496.31293",0.01127,0.01131,-0.12400,-0.35322,2023-10-21 21:44:03,2023-10-21 20:42:33
6,OP,Optimism,optimism,"711,871,150.28026","710,777,993.87818",0.01066,0.01067,0.15380,-0.07606,2023-10-21 21:44:03,2023-10-21 20:42:33
7,AVAX,Avalanche,avalanche-2,"664,029,422.81139","665,869,604.39097",0.00994,0.00999,-0.27636,-0.50523,2023-10-21 21:44:03,2023-10-21 20:42:33
8,0,Base,0,"317,117,558.32416","316,259,103.35082",0.00475,0.00475,0.27144,0.04131,2023-10-21 21:44:03,2023-10-21 20:42:33
9,CRO,Cronos,crypto-com-chain,"301,746,701.40242","302,204,490.23983",0.00452,0.00454,-0.15148,-0.38064,2023-10-21 21:44:03,2023-10-21 20:42:33


<font size="5">TVL_CHANGE</font>

In [12]:
tvl_per_chain_with_threshold = tvl_per_chain[tvl_per_chain.tvl >= dp.string_number_to_integer(tvl_per_chain_threshold)]

In [13]:
tvl_per_chain_with_threshold.head(10).reset_index(drop=True)

,tokenSymbol,name,gecko_id,tvl,tvl_y,share,share_y,tvl_perc,share_perc,date,date_y
0,ELA,Elastos,elastos,"1,745,383.77074","1,161,243.40977",0.00003,0.00002,50.30301,49.95805,2023-10-21 21:44:03,2023-10-21 20:42:33
1,MC,Beam,0,"1,430,633.45726","1,408,691.49926",0.00002,0.00002,1.55761,1.32453,2023-10-21 21:44:03,2023-10-21 20:42:33
2,OKT,OKExChain,oec-token,"98,000,703.39410","96,663,282.64256",0.00147,0.00145,1.38359,1.15090,2023-10-21 21:44:03,2023-10-21 20:42:33
3,JUNO,Juno,juno-network,"845,869.60449","835,703.27572",0.00001,0.00001,1.21650,0.98420,2023-10-21 21:44:03,2023-10-21 20:42:33
4,WEMIX,WEMIX,wemix-token,"9,472,814.48971","9,370,184.27768",0.00014,0.00014,1.09528,0.86326,2023-10-21 21:44:03,2023-10-21 20:42:33
5,NLS,Nolus,nolus,"1,215,959.48852","1,203,502.76100",0.00002,0.00002,1.03504,0.80316,2023-10-21 21:44:03,2023-10-21 20:42:33
6,KUJI,Kujira,kujira,"38,286,228.15172","37,917,816.86284",0.00057,0.00057,0.97160,0.73987,2023-10-21 21:44:03,2023-10-21 20:42:33
7,MTRG,Meter,meter,"6,650,523.72467","6,586,652.27803",0.00010,0.00010,0.96971,0.73798,2023-10-21 21:44:03,2023-10-21 20:42:33
8,0,NOS,0,"2,051,443.51070","2,035,037.15980",0.00003,0.00003,0.80619,0.57484,2023-10-21 21:44:03,2023-10-21 20:42:33
9,FLR,Flare,flare-networks,"15,364,334.45885","15,245,761.51949",0.00023,0.00023,0.77774,0.54645,2023-10-21 21:44:03,2023-10-21 20:42:33


<font size="6">STABLECOINS</font>

In [14]:
save_in_db = True
circulation_threshold = '500 000'

In [15]:
stablecoins = defi_stats.StablecoinsStats(save_in_db, ).stats

2023-10-21 21:44:04. StablecoinsStats: 2023-10-21 20:42:34 - 2023-10-21 21:44:04.
2023-10-21 21:44:04. Timedelta: 0 days 1 hours 1 minutes 30 seconds.
2023-10-21 21:44:04. StablecoinsStats: current market data saved in db.
2023-10-21 21:44:04. StablecoinsStats: stats saved in db.
2023-10-21 21:44:04. Report created in: 0:00:00.601547.


In [16]:
stablecoins.head()

,symbol,name,gecko_id,price,price_y,circulating,circulating_y,price_perc,circ_perc,chains,date,date_y
0,USDT,Tether,tether,1.00000,1.00000,"84,131,359,672.00000","84,127,259,672.00000",0.00000,0.00487,"[Tron, Ethereum, BSC, Arbitrum, Solana, Avalan...",2023-10-21 21:44:04,2023-10-21 20:42:34
1,USDC,USD Coin,usd-coin,0.99959,1.00000,"24,938,123,191.00000","24,938,141,663.00000",-0.04100,-0.00007,"[Ethereum, Arbitrum, Solana, Polygon, Tron, BS...",2023-10-21 21:44:04,2023-10-21 20:42:34
2,DAI,Dai,dai,0.99951,0.99984,"5,548,254,013.00000","5,549,282,335.00000",-0.03211,-0.01853,"[Ethereum, Polygon, Arbitrum, Gnosis, BSC, Fan...",2023-10-21 21:44:04,2023-10-21 20:42:34
3,TUSD,TrueUSD,true-usd,0.99875,1.00000,"3,335,533,069.00000","3,335,533,069.00000",-0.12520,0.00000,"[Tron, Ethereum, BSC, Avalanche, Fantom, Polyg...",2023-10-21 21:44:04,2023-10-21 20:42:34
4,BUSD,Binance USD,binance-usd,1.00100,1.00200,"2,097,301,100.00000","2,097,301,100.00000",-0.09980,0.00000,"[Ethereum, BSC, Kava, Avalanche, Polygon, IoTe...",2023-10-21 21:44:04,2023-10-21 20:42:34


<font size="5">CIRCULATION_PERC</font>

In [17]:
stablecoins_with_threshold = stablecoins[stablecoins.circulating >= dp.string_number_to_integer(circulation_threshold)]

In [18]:
circ_perc = stablecoins_with_threshold.sort_values('circ_perc', ascending=False).reset_index(drop=True)

In [19]:
circ_perc.head()

,symbol,name,gecko_id,price,price_y,circulating,circulating_y,price_perc,circ_perc,chains,date,date_y
0,USK,USK,usk,0.99796,1.00100,"2,333,639.00000","2,316,872.00000",-0.30420,0.72369,[Kujira],2023-10-21 21:44:04,2023-10-21 20:42:34
1,GHO,GHO,gho,0.97009,0.97159,"24,404,634.00000","24,355,502.00000",-0.15398,0.20173,[Ethereum],2023-10-21 21:44:04,2023-10-21 20:42:34
2,NOTE,Note,note,0.99830,0.99898,"15,911,246.00000","15,903,259.00000",-0.06847,0.05022,[Canto],2023-10-21 21:44:04,2023-10-21 20:42:34
3,crvUSD,crvUSD,crvusd,0.99773,0.99856,"114,001,636.00000","113,962,823.00000",-0.08392,0.03406,[Ethereum],2023-10-21 21:44:04,2023-10-21 20:42:34
4,IUSD,iUSD,iusd,0.94948,0.94564,"13,087,087.00000","13,085,302.00000",0.40639,0.01364,[Cardano],2023-10-21 21:44:04,2023-10-21 20:42:34


In [20]:
circ_perc.tail()

,symbol,name,gecko_id,price,price_y,circulating,circulating_y,price_perc,circ_perc,chains,date,date_y
65,USDN,Neutrino USD,neutrino,0.09195,0.09144,"86,759,001.00000","86,761,658.00000",0.56432,-0.00306,"[Waves, Ethereum, BSC, Polygon]",2023-10-21 21:44:04,2023-10-21 20:42:34
66,DAI,Dai,dai,0.99951,0.99984,"5,548,254,013.00000","5,549,282,335.00000",-0.03211,-0.01853,"[Ethereum, Polygon, Arbitrum, Gnosis, BSC, Fan...",2023-10-21 21:44:04,2023-10-21 20:42:34
67,USDH,USDH,usdh,0.98811,1.00400,"3,530,764.00000","3,536,387.00000",-1.58297,-0.15900,[Solana],2023-10-21 21:44:04,2023-10-21 20:42:34
68,SUSD,sUSD,nusd,0.99796,0.99991,"76,939,365.00000","77,148,402.00000",-0.19502,-0.27095,"[Optimism, Ethereum, Arbitrum, Fantom, Ontology]",2023-10-21 21:44:04,2023-10-21 20:42:34
69,GRAI,GRAI,grai,0.98614,0.98546,"13,847,377.00000","13,925,849.00000",0.06941,-0.56350,"[Ethereum, Arbitrum, Polygon zkEVM, Optimism, ...",2023-10-21 21:44:04,2023-10-21 20:42:34
